In [90]:
import pandas as pd
import numpy as np

In [91]:
#Load data
df_raw = pd.read_csv("train.csv")
print(df_raw)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
5        6          50       RL         85.0    14115   Pave   NaN      IR1   
6        7          20       RL         75.0    10084   Pave   NaN      Reg   
7        8          60       RL          NaN    10382   Pave   NaN      IR1   
8        9          50       RM         51.0     6120   Pave   NaN      Reg   
9       10         190       RL         50.0     7420   Pave   NaN      Reg   
10      11          20       RL         70.0    11200   Pave   NaN      Reg   
11      12          60       RL         85.0    1192

In [92]:
# EDA
# 1) Describe each variable in the dataset.
print(df_raw.columns.tolist())
with open("EDA\\column_analysis.txt", 'a') as f:
    for col in df_raw.columns.tolist():
        f.write("Analysis of column {}".format(col))
        f.write("\n")
        f.write(df_raw[col].describe().to_string())
        f.write("\n")
        f.write("Missing values: {}".format(df_raw[col].isnull().sum()))
        f.write("\n")
        f.write("Column data type: {}".format(df_raw[col].dtype))
        f.write("\n")
        f.write("Count of individual values in column: ")
        f.write("\n")
        f.write(df_raw.groupby([col]).size().sort_values(ascending=False).to_string())
        f.write("\n")
        f.write("\n")
# 2) Do plots


['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC'

In [93]:
# Test/Train split
train_split = 0.8
df_num_rows = len(df_raw.index)
num_train_rows = int(round(df_num_rows * train_split, 0))
df_train = df_raw[0:num_train_rows]
df_test = df_raw[num_train_rows:]
print(len(df_train.index))
print(len(df_test.index))

2335
584


In [94]:
# Preprocessing
# 1) Get numeric and categorical cols
all_cols = df_raw.columns.tolist()
numeric_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = [col for col in all_cols if col not in numeric_cols]
cols_to_drop = ["Id"]
numeric_cols = [col for col in numeric_cols if col not in cols_to_drop]
categorical_cols = [col for col in categorical_cols if col not in cols_to_drop]

# 2) Fill NA numeric rows with mean
df_train = df_train.fillna(df_train.mean())
df_test = df_test.fillna(df_test.mean())

# 3) Get categorical cols by cardinality

# 4) Normalize and standardize numeric cols:
reg_params = {}
for col in numeric_cols:
    reg_params[col] = {
        "mean": df_raw[col].mean(),
        "std": df_raw[col].std(),
        "max": df_raw[col].max(),
        "min": df_raw[col].min()
    }
    df_train[col] = (df_train[col] - df_train[col].mean())/ df_train[col].std()
    df_train[col] = (df_train[col] - df_train[col].min())/(df_train[col].max() - df_train[col].min())

# 4) One hot encode categorical variables
for col in categorical_cols:
    df_train[col] = pd.Categorical(df_train[col])
    onehot_cols = pd.get_dummies(df_train[col], prefix = col)
    df_train = df_train.join(onehot_cols, how="inner")

# 5) Drop not one-hot encoded categorical cols.
all_cols = df_train.columns.tolist()
cols_train = [col for col in all_cols if col not in categorical_cols]
df_train = df_train[cols_train]


    






In [96]:

print(cols_train)
# df_train.to_excel("df_onehot.xlsx")

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_AllPub', 'Utilities_NoSeWa', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Bl